In [ ]:
!pip install -U diffusers transformers ftfy gradio accelerate

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
import inspect
from typing import List, Optional, Union
import numpy as np
import torch
import os
import PIL

from diffusers import StableDiffusionInpaintPipeline
import requests
from io import BytesIO

!wget https://raw.githubusercontent.com/huggingface/diffusers/main/examples/research_projects/dreambooth_inpaint/train_dreambooth_inpaint.py

In [ ]:
#enter your correct huggingface token
from huggingface_hub import login
login('<HuggingFace Token>', add_to_git_credential=True)

In [ ]:
pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-inpainting",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)
pipe.enable_attention_slicing()

# Move the pipeline to the correct device (GPU if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = pipe.to(device)

In [ ]:
!accelerate launch train_dreambooth_inpaint.py \
    --pretrained_model_name_or_path="stabilityai/stable-diffusion-2-inpainting"  \
    --instance_data_dir="images/Images_jpg" \
    --output_dir="stable-diffusion-inpainting-painting" \
    --instance_prompt="old painting" \
    --resolution=256 \
    --mixed_precision="fp16" \
    --train_batch_size=1 \
    --learning_rate=5e-6 \
    --lr_scheduler="constant" \
    --lr_warmup_steps=0 \
    --max_train_steps=500 \
    --gradient_accumulation_steps=2 \
    --gradient_checkpointing \
    --train_text_encoder \
    --seed="0" \
    --push_to_hub